# Note to the grader:
As mentioned in our report, this code was sourced from a publicly available GitHub repository for research purposes. The repository can be accesed via https://github.com/RishaRane/Sarcasm_Detection_using_BERT. The code was appropriately updated and adapted to process our dataset for comparative analysis. To ensure a scientifically sound comparison, we also adjusted relevant hyperparameters, such as the number of LSTMs and training epochs.

We want to clarify that there was no intention to take credit for someone else's work.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Sarcasm Detection using Hierarchical Bert

## 1. DATA LOADING AND PREPROCESSING

In [ ]:
#importing libraries
import zipfile
import pandas as pd
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [ ]:
training = pd.read_csv("/content/drive/MyDrive/FALL2024/comp550/final_project/sarcasm/reddit/sarcasm_detection_reddit_training.csv")
testing = pd.read_csv("/content/drive/MyDrive/FALL2024/comp550/final_project/sarcasm/reddit/sarcasm_detection_reddit_testing.csv")
df= pd.concat([training, testing], axis=0)
df['label'] = df['label'].map({'SARCASM': 1, 'NOT_SARCASM': 0})
df = df[['response', 'label']]
df.head()

,response,label
0,"Yeah I mean there's only one gender anyways, w...",1
1,"Sounds like you don't like science, you theist...",1
2,"Ofc play them in try mode, Blizzard were so ge...",1
3,"I don't understand, Reddit told me that Hillar...",1
4,"yeh, they're the reigning triple premiers, why...",1


In [ ]:
#removing unwanted numerals and symbols
df['response'] = df['response'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

In [ ]:
#converting the data into lowercase
def lowercase(text):
  return text.lower()

df['response'] = df['response'].apply(lowercase)

## 2. TOKENIZATION

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
#function for tokenization
def tokenize_data(text, max_length = 100):
  return tokenizer(
      text.tolist(),
      max_length = max_length,
      truncation = True,
      padding = 'max_length',
      return_tensors = 'np'
  )

tokenized_data = tokenize_data(df['response'])

In [ ]:
tokenized_data

{'input_ids': array([[  101,  3398,  1045, ...,     0,     0,     0],
       [  101,  4165,  2066, ...,     0,     0,     0],
       [  101,  1997,  2278, ...,     0,     0,     0],
       ...,
       [  101, 11798,  3122, ...,     0,     0,     0],
       [  101,  2020,  2025, ...,     0,     0,     0],
       [  101,  2003,  2016, ...,     0,     0,     0]]), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}

## 3. TRAIN_TEST_SPLIT

In [ ]:
X = tokenized_data['input_ids']
y = df['label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
X_train.shape, X_test.shape

((4960, 100), (1240, 100))

## 4.Building the model according to the proposed architecture

In [ ]:
class HierarchicalBERT(tf.keras.Model):
    def __init__(self, bert_model, lstm_units, cnn_filters, dense_units):
        super(HierarchicalBERT, self).__init__()
        self.bert = bert_model

        # Sentence Encoding Layer
        self.dense_sentence = tf.keras.layers.Dense(768, activation='relu')

        # Context Summarization Layer
        self.mean_pooling = tf.keras.layers.GlobalAveragePooling1D()

        # Context Encoder Layer
        self.bilstm_encoder = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences=True))

        # CNN Layer
        self.conv = tf.keras.layers.Conv1D(cnn_filters, 2, activation='relu')
        self.pool = tf.keras.layers.GlobalMaxPooling1D()

        # Fully Connected Layer
        self.dense_fc = tf.keras.layers.Dense(dense_units, activation='relu')
        self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        # BERT Embeddings
        bert_output = self.bert(inputs)[0]  # (batch_size, seq_len, hidden_size)

        # Sentence Encoding Layer
        sentence_encoded = self.dense_sentence(bert_output)  # (batch_size, seq_len, 768)

        # Context Summarization Layer
        context_summarized = self.mean_pooling(sentence_encoded)  # (batch_size, 768)

        # Expand dimensions to match the input shape required by LSTM
        context_summarized = tf.expand_dims(context_summarized, 1)  # (batch_size, 1, 768)

        # Context Encoder Layer
        context_encoded = self.bilstm_encoder(context_summarized)  # (batch_size, 1, 2 * lstm_units)

        # Squeeze the second dimension to match the input shape required by Conv1D
        context_encoded_squeezed = tf.squeeze(context_encoded, axis=1)  # (batch_size, 2 * lstm_units)

        # Add the channels dimension to match the input shape required by Conv1D
        context_encoded_expanded = tf.expand_dims(context_encoded_squeezed, axis=-1)  # (batch_size, 2 * lstm_units, 1)

        # CNN Layer
        conv_output = self.conv(context_encoded_expanded)  # (batch_size, new_seq_len, cnn_filters)
        pooled_output = self.pool(conv_output)  # (batch_size, cnn_filters)

        # Fully Connected Layer
        dense_output = self.dense_fc(pooled_output)  # (batch_size, dense_units)

        # Output Layer
        final_output = self.output_layer(dense_output)  # (batch_size, 1)
        return final_output


In [ ]:
#loading the pretrained BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
#defining the hierarchical bert model
model = HierarchicalBERT(bert_model, lstm_units = 300, cnn_filters = 64, dense_units = 32)

In [ ]:
model.compile(optimizer = 'adamW',loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs = 25, batch_size = 32)

Epoch 1/25


155/155 [==============================] - 142s 664ms/step - loss: 0.6940 - accuracy: 0.4980
Epoch 2/25
155/155 [==============================] - 103s 665ms/step - loss: 0.6948 - accuracy: 0.5020
Epoch 3/25
155/155 [==============================] - 103s 664ms/step - loss: 0.6937 - accuracy: 0.4956
Epoch 4/25
155/155 [==============================] - 103s 663ms/step - loss: 0.6940 - accuracy: 0.4923
Epoch 5/25
155/155 [==============================] - 102s 661ms/step - loss: 0.6938 - accuracy: 0.4935
Epoch 6/25
155/155 [==============================] - 102s 660ms/step - loss: 0.6938 - accuracy: 0.5004
Epoch 7/25
155/155 [==============================] - 103s 663ms/step - loss: 0.6936 - accuracy: 0.5012
Epoch 8/25
155/155 [==============================] - 103s 664ms/step - loss: 0.6941 - accuracy: 0.4883
Epoch 9/25
155/155 [==============================] - 103s 662ms/step - loss: 0.6938 - accuracy: 0.4815
Epoch 10/25
155/155 [==============================] - 103s 664ms/step - lo

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Model Accuracy : {accuracy * 100}')

39/39 [==============================] - 13s 214ms/step - loss: 0.6933 - accuracy: 0.4919
Model Accuracy : 49.193549156188965
